In [3]:
import sys

sys.path.append('..')
from config import *
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 数据库设计文档生成器（逻辑模型物理模型同时输出）

In [4]:
BIG_PROMPT = r"""
你是数据库设计文档生成器。根据我提供的 MySQL 建表 SQL（CREATE TABLE），为每张表生成两张 Markdown 表格：

1）逻辑模型表
2）物理模型表

只输出：标题 + 表格，不要多余说明文字。

全局要求：
- 语言：中文
- 每张表依次输出：① 逻辑模型表 ② 物理模型表
- 不要增删字段，不要省略字段
- 有 COMMENT 用 COMMENT 作为中文名/说明；没有就根据字段名简单推断，看不出就留空
- 当内容不确定或无法从 SQL 明确判断时，对应单元格留空，不要填「待确认」「—」等占位词

一、逻辑模型表

标题格式：
### 表 X-X [表中文名称] 逻辑模型（表名：xxx）

表头固定为：
| 序号 | 逻辑属性名 | 业务含义 | 逻辑数据类型 | 是否必填 | 取值范围/业务约束 | 备注 |

填写规则：
- 逻辑属性名：字段的中文名，优先使用 COMMENT；无 COMMENT 时可根据字段名简单推断，看不出就留空
- 业务含义：一句话说明字段用途，看不出就留空
- 逻辑数据类型：归类为「字符串 / 整数 / 数值 / 日期时间 / 布尔 / 枚举」等，按最接近类型填写，极端不确定时留空
- 是否必填：主键或 NOT NULL = 是，其它 = 否
- 取值范围/业务约束：如「唯一」「枚举：M, F」等，没有就留空
- 备注：如「逻辑主键」「系统生成」等，不确定就留空

二、物理模型表

标题格式：
### 表 X-X [表中文名称] 物理模型（表名：xxx）

表头固定为：
| 序号 | 字段名 | 数据类型 | 长度/精度 | 是否为空 | 主/外键及约束 | 索引情况 | 字段说明 |

填写规则：
- 数据类型：按 SQL 中数据类型原样填写，如 VARCHAR、INT、BIGINT、DATETIME、DECIMAL 等
- 长度/精度：VARCHAR(20) → 写 20，DECIMAL(10,2) → 写 10,2，无长度或精度时留空
- 是否为空：NOT NULL → 不可为空，其它 → 可为空
- 主/外键及约束：如「主键」「AUTO_INCREMENT」「UNIQUE」「外键 -> 表(字段)」等；没有或无法确定时留空
- 索引情况：如「主键索引」「唯一索引 uk_xxx」「普通索引 idx_xxx」，没有或无法确定时留空
- 字段说明：简短中文说明，优先使用 COMMENT；无 COMMENT 时可根据字段名简单推断，看不出就留空

现在请根据我接下来提供的建表 SQL，按以上规范为每一张表生成对应的【逻辑模型表】和【物理模型表】。
接下来是需要你转换的建表 SQL：
{sql_ddl}
"""
prompt = PromptTemplate(
    input_variables=["sql_ddl"],
    template=BIG_PROMPT
)

llm = ChatOpenAI(model="qwen-plus")

chain = prompt | llm

In [7]:
sql_ddl = """
          create table address
          (
              address_id  varchar(256) not null
                  primary key,
              customer_id varchar(256) null,
              is_default  tinyint(1)                          null,
              first_name  varchar(512) null,
              phone       varchar(512) null,
              last_name   varchar(512) null,
          ) comment '客户信息表' row_format = DYNAMIC; \
          """

In [8]:
result = chain.invoke({"sql_ddl": sql_ddl})
from IPython.display import Markdown, display

display(Markdown(result.text))

### 表 1-1 客户信息表 逻辑模型（表名：address）

| 序号 | 逻辑属性名 | 业务含义 | 逻辑数据类型 | 是否必填 | 取值范围/业务约束 | 备注 |
|------|------------|--------|--------------|----------|--------------------|------|
| 1 | 地址编号 | 唯一标识一个地址 | 字符串 | 是 | 唯一 | 逻辑主键 |
| 2 | 客户编号 | 关联客户 | 字符串 | 否 |  |  |
| 3 | 是否默认地址 | 标识是否为默认收货地址 | 布尔 | 否 |  |  |
| 4 | 名 | 客户姓名中的名 | 字符串 | 否 |  |  |
| 5 | 电话 | 客户联系电话 | 字符串 | 否 |  |  |
| 6 | 姓 | 客户姓名中的姓 | 字符串 | 否 |  |  |

### 表 1-1 客户信息表 物理模型（表名：address）

| 序号 | 字段名 | 数据类型 | 长度/精度 | 是否为空 | 主/外键及约束 | 索引情况 | 字段说明 |
|------|--------|----------|-----------|----------|----------------|----------|----------|
| 1 | address_id | varchar | 256 | 不可为空 | 主键 | 主键索引 | 地址编号 |
| 2 | customer_id | varchar | 256 | 可为空 |  |  | 客户编号 |
| 3 | is_default | tinyint | 1 | 可为空 |  |  | 是否默认地址 |
| 4 | first_name | varchar | 512 | 可为空 |  |  | 名 |
| 5 | phone | varchar | 512 | 可为空 |  |  | 电话 |
| 6 | last_name | varchar | 512 | 可为空 |  |  | 姓 |

# 分开输出逻辑模型和物理模型
## 逻辑模型

In [16]:
BIG_PROMPT = r"""
你是数据库逻辑模型文档生成器。根据我提供的 MySQL 建表 SQL（CREATE TABLE），为每张表生成一张逻辑模型 Markdown 表格。

输出要求：
- 语言：中文
- 只输出一张 Markdown 表格，不要输出标题、解释说明或其他文字
- 不要输出任何代码块标记（不要输出 ```）
- 不要增删字段，不要省略字段
- 每张表仅输出一张逻辑模型表格；如果 SQL 中有多张表，则按顺序依次输出多张表格（表格之间空一行）
- 有 COMMENT 用 COMMENT 作为中文名/说明；没有就根据字段名简单推断，看不出就留空
- 当内容不确定或无法从 SQL 明确判断时，对应单元格留空，不要填「待确认」「—」等占位词

逻辑模型表的表头固定为：

| 序号 | 逻辑属性名 | 业务含义 | 逻辑数据类型 | 是否必填 | 取值范围/业务约束 | 备注 |

填写规则：
- 序号：从 1 开始，按字段在表中的顺序依次递增
- 逻辑属性名：字段的中文名，优先使用 COMMENT；无 COMMENT 时可根据字段名简单推断，看不出就留空
- 业务含义：一句话说明字段用途，看不出就留空
- 逻辑数据类型：归类为「字符串 / 整数 / 数值 / 日期时间 / 布尔 / 枚举」等，按最接近类型填写，极端不确定时留空
- 是否必填：主键或 NOT NULL = 是，其它 = 否
- 取值范围/业务约束：如「唯一」「枚举：M, F」「大于 0」等，没有就留空
- 备注：如「逻辑主键」「系统生成」等，不确定就留空

下面是需要你转换的建表 SQL：
{sql_ddl}
"""
prompt = PromptTemplate(
    input_variables=["sql_ddl"],
    template=BIG_PROMPT
)

llm = ChatOpenAI(model="qwen-plus")

chain = prompt | llm

In [17]:
sql_ddl = """
          create table address
          (
              address_id  varchar(256) not null
                  primary key,
              customer_id varchar(256) null,
              is_default  tinyint(1)                          null,
              first_name  varchar(512) null,
              phone       varchar(512) null,
              last_name   varchar(512) null,
          ) comment '客户信息表' row_format = DYNAMIC; \
          """

result = chain.invoke({"sql_ddl": sql_ddl})
from IPython.display import Markdown, display

display(Markdown(result.text))

| 序号 | 逻辑属性名 | 业务含义 | 逻辑数据类型 | 是否必填 | 取值范围/业务约束 | 备注 |
|------|----------|--------|------------|--------|------------------|------|
| 1 | 地址编号 | 地址的唯一标识 | 字符串 | 是 | 唯一 | 逻辑主键 |
| 2 | 客户编号 | 关联的客户ID | 字符串 | 否 | | |
| 3 | 是否默认 | 标识是否为默认地址 | 布尔 | 否 | | |
| 4 | 名字 | 客户名字 | 字符串 | 否 | | |
| 5 | 电话 | 客户联系电话 | 字符串 | 否 | | |
| 6 | 姓氏 | 客户姓氏 | 字符串 | 否 | | |

In [15]:
from docx import Document

md_table = result.text


def parse_md_table(md: str):
    lines = [l.strip() for l in md.strip().splitlines() if l.strip()]
    # 去掉分隔行（第二行那一堆 ---）
    header = [c.strip() for c in lines[0].strip('|').split('|')]
    data_lines = [l for l in lines[2:]]
    rows = [[c.strip() for c in line.strip('|').split('|')] for line in data_lines]
    return header, rows


header, rows = parse_md_table(md_table)

doc = Document()

# 创建表格：1 行表头 + 数据行
table = doc.add_table(rows=1 + len(rows), cols=len(header))
table.style = 'Table Grid'  # 可根据需要改样式

# 写表头
hdr_cells = table.rows[0].cells
for i, text in enumerate(header):
    hdr_cells[i].text = text

# 写数据
for r_i, row in enumerate(rows, start=1):
    row_cells = table.rows[r_i].cells
    for c_i, text in enumerate(row):
        row_cells[c_i].text = text

doc.save('student_table.docx')

## 物理模型

In [18]:
BIG_PROMPT = r"""
你是数据库物理模型文档生成器。根据我提供的 MySQL 建表 SQL（CREATE TABLE），为每张表生成一张物理模型 Markdown 表格。

输出要求：
- 语言：中文
- 只输出一张 Markdown 表格，不要输出标题、解释说明或其他文字
- 不要输出任何代码块标记（不要输出 ```）
- 不要增删字段，不要省略字段
- 每张表仅输出一张物理模型表格；如果 SQL 中有多张表，则按顺序依次输出多张表格（表格之间空一行）
- 有 COMMENT 用 COMMENT 作为中文名/说明；没有就根据字段名简单推断，看不出就留空
- 当内容不确定或无法从 SQL 明确判断时，对应单元格留空，不要填「待确认」「—」等占位词

物理模型表的表头固定为：

| 序号 | 字段名 | 数据类型 | 长度/精度 | 是否为空 | 主/外键及约束 | 索引情况 | 字段说明 |

填写规则：
- 序号：从 1 开始，按字段在表中的顺序依次递增
- 字段名：直接填写 SQL 中的字段名
- 数据类型：按 SQL 中的数据类型原样填写，如 VARCHAR、INT、BIGINT、DATETIME、DECIMAL 等（不含长度）
- 长度/精度：VARCHAR(20) → 写 20，DECIMAL(10,2) → 写 10,2；无长度或精度时留空
- 是否为空：NOT NULL → 不可为空，其它 → 可为空
- 主/外键及约束：如「主键」「AUTO_INCREMENT」「UNIQUE」「外键 -> 关联表(字段)」等；没有或无法确定时留空
- 索引情况：如「主键索引」「唯一索引 uk_xxx」「普通索引 idx_xxx」，没有或无法确定时留空
- 字段说明：简短中文说明，优先使用 COMMENT；无 COMMENT 时可根据字段名简单推断，看不出就留空

下面是需要你转换的建表 SQL：
{sql_ddl}
"""
prompt = PromptTemplate(
    input_variables=["sql_ddl"],
    template=BIG_PROMPT
)

llm = ChatOpenAI(model="qwen-plus")

chain = prompt | llm
sql_ddl = """
          create table address
          (
              address_id  varchar(256) not null
                  primary key,
              customer_id varchar(256) null,
              is_default  tinyint(1)                          null,
              first_name  varchar(512) null,
              phone       varchar(512) null,
              last_name   varchar(512) null,
          ) comment '客户信息表' row_format = DYNAMIC; \
          """

result = chain.invoke({"sql_ddl": sql_ddl})
from IPython.display import Markdown, display

display(Markdown(result.text))

| 序号 | 字段名 | 数据类型 | 长度/精度 | 是否为空 | 主/外键及约束 | 索引情况 | 字段说明 |
|------|--------|----------|-----------|----------|----------------|----------|----------|
| 1 | address_id | varchar | 256 | 不可为空 | 主键 | 主键索引 | 地址ID |
| 2 | customer_id | varchar | 256 | 可为空 |  |  | 客户ID |
| 3 | is_default | tinyint | 1 | 可为空 |  |  | 是否默认地址 |
| 4 | first_name | varchar | 512 | 可为空 |  |  | 名 |
| 5 | phone | varchar | 512 | 可为空 |  |  | 电话 |
| 6 | last_name | varchar | 512 | 可为空 |  |  | 姓 |